In [1]:
%matplotlib qt5
import numpy as np
from sklearn.linear_model import LinearRegression
from scipy.ndimage.interpolation import shift
from scipy.signal import decimate
from matplotlib import pyplot as plt
from scipy import stats
import pandas as pd
import pickle


In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The savefig.frameon rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.level rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.
In C:\Users\francescag\AppData\Local\Continuum\miniconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The verbose.fileo rcparam was deprecated in Matplotlib 3.1 and will be removed in 3.3.


In [2]:
def rolling_zscore(x, window=10*10000):
    r = x.rolling(window=window)
    m = r.mean().shift(1)
    s = r.std(ddof=0).shift(1)
    z = (x-m)/s
    return z

In [62]:
mouse = 'SNL_photo19'
date = '20200317'
saving_folder = 'W:\\photometry_2AC\\processed_data\\' + mouse + '\\'
restructured_data_filename = mouse + '_' + date + '_' + 'restructured_data.pkl'
trial_data = pd.read_pickle(saving_folder + restructured_data_filename) 
dff_trace_filename = mouse + '_' + date + '_' + 'smoothed_signal.npy'
dff = np.load(saving_folder + dff_trace_filename)

In [63]:
window_size_seconds = 10
sample_rate = 10000
decimate_factor = 100

In [64]:
rolling_zscored_dff = rolling_zscore(pd.Series(dff), window=window_size_seconds*sample_rate)

In [65]:
rolling_zscored_dff.shape

(18002500,)

In [66]:
downsampled_zscored_dff = decimate(decimate(rolling_zscored_dff[window_size_seconds*sample_rate:], int(decimate_factor/10)), int(decimate_factor/10))

In [8]:
plt.plot(downsampled_zscored_dff)

In [67]:
num_samples = downsampled_zscored_dff.shape[0]

In [68]:
def turn_timestamps_into_continuous(num_samples, *behavioural_events):
    continuous_parameters = []
    for event_type_timestamps in behavioural_events:
        continuous_time_version = np.zeros([num_samples])
        continuous_time_version[event_type_timestamps] = 1
        continuous_parameters.append(continuous_time_version)
    return continuous_parameters

In [69]:
aligned_filename = mouse + '_' + date + '_' + 'behavioural_events.p'
save_filename = saving_folder + aligned_filename
example_session_data = pickle.load(open(save_filename, "rb"))

In [70]:
def convert_behavioural_timestamps_into_samples(time_stamps, window_to_remove, sample_rate=10000, decimate_factor=100):
    adjusted_stamps = (time_stamps - window_to_remove)*sample_rate/decimate_factor
    adjusted_stamps = np.round(np.vstack(adjusted_stamps).astype(np.float)).astype(int)
    return adjusted_stamps

In [71]:
data = example_session_data.reward_data.contra_data.event_times
convert_behavioural_timestamps_into_samples(data, window_size_seconds)


array([[   283],
       [  1040],
       [  1721],
       [  2361],
       [  3198],
       [  3668],
       [  8964],
       [  9289],
       [  9784],
       [ 10040],
       [ 10600],
       [ 11185],
       [ 11466],
       [ 11914],
       [ 12456],
       [ 12762],
       [ 13263],
       [ 13541],
       [ 13829],
       [ 15164],
       [ 15623],
       [ 16139],
       [ 24190],
       [ 24766],
       [ 25061],
       [ 25322],
       [ 25654],
       [ 25952],
       [ 26334],
       [ 27041],
       [ 27401],
       [ 28006],
       [ 29208],
       [ 29778],
       [ 30837],
       [ 31444],
       [ 31676],
       [ 32705],
       [ 32957],
       [ 33232],
       [ 34037],
       [ 34320],
       [ 35198],
       [ 35902],
       [ 36293],
       [ 36546],
       [ 36818],
       [ 37461],
       [ 37967],
       [ 38210],
       [ 38743],
       [ 39448],
       [ 39671],
       [ 39940],
       [ 40495],
       [ 41045],
       [ 41535],
       [ 41794],
       [ 42855

In [72]:
ipsi_choices = convert_behavioural_timestamps_into_samples(example_session_data.choice_data.ipsi_data.event_times, window_size_seconds)
contra_choices = convert_behavioural_timestamps_into_samples(example_session_data.choice_data.contra_data.event_times, window_size_seconds)
high_cues = convert_behavioural_timestamps_into_samples(example_session_data.cue_data.high_cue_data.event_times, window_size_seconds)
low_cues = convert_behavioural_timestamps_into_samples(example_session_data.cue_data.low_cue_data.event_times, window_size_seconds)
ipsi_rewards = convert_behavioural_timestamps_into_samples(example_session_data.reward_data.ipsi_data.event_times, window_size_seconds)
contra_rewards = convert_behavioural_timestamps_into_samples(example_session_data.reward_data.contra_data.event_times, window_size_seconds)

In [73]:
#all_cues = np.concatenate([ipsi_cues, contra_cues])
all_rewards = np.concatenate([ipsi_rewards, contra_rewards])

In [74]:
parameters = turn_timestamps_into_continuous(num_samples, ipsi_choices, contra_choices, high_cues, low_cues, ipsi_rewards, contra_rewards)

In [18]:
#parameters = turn_timestamps_into_continuous(num_samples, ipsi_choices, contra_choices, all_cues, all_rewards)

In [75]:
def make_design_matrix(parameters, window_min=-1*10000/100, window_max=2*10000/100):
    num_parameters = len(parameters)
    shifts = np.arange(window_min, window_max + 1)
    shift_window_size = shifts.shape[0]
    X = np.zeros([parameters[0].shape[0], shift_window_size*num_parameters])
    all_param_indices = []
    for shift_num, shift_val in  enumerate(shifts):
        for param_num, param in enumerate(parameters):
            param_indices = range(param_num*shift_window_size, param_num*shift_window_size + shift_window_size)
            all_param_indices.append(param_indices)
            shifted_param = shift(param, shift_val, cval=0) 
            X[:, param_indices[shift_num]] = shifted_param
    return(all_param_indices, X)


In [76]:
all_param_indices, X = make_design_matrix(parameters)

In [77]:
results = LinearRegression().fit(X, downsampled_zscored_dff)

In [78]:
def plot_kernels(parameter_names, regression_results,window_min=-1*10000/100, window_max=2*10000/100):
    
    fig, axs = plt.subplots(nrows=1, ncols=len(parameter_names), sharey=True, figsize=(15,8))
    axs[0].set_ylabel('Regression coefficient')
    shifts = np.arange(window_min, window_max + 1)/100
    shift_window_size = shifts.shape[0]
    for param_num, param_name in enumerate(parameter_names):
        param_kernel = regression_results.coef_[param_num*shift_window_size:(param_num+1)*shift_window_size]
        axs[param_num].plot(shifts, param_kernel, label=param_name)
        axs[param_num].set_title(param_name)
        axs[param_num].axvline([0])
        axs[param_num].set_xlabel('Time (s)')

In [79]:
param_names = ['ipsi_choices',' contra_choices', 'high_cues', 'low_cues', 'ipsi_rewards',' contra_rewards']
plot_kernels(param_names, results)

In [61]:
plot_filename = mouse + '_' + date + '_' + 'linear_regresion_kernels.pdf' 
plt.savefig(saving_folder + plot_filename, transparent=True, optimize=True)

In [151]:
def save_kernels(parameter_names, regression_results,  window_min=-1 * 10000 / 100, window_max=2 * 10000 / 100):

    shifts = np.arange(window_min, window_max + 1) / 100
    shift_window_size = shifts.shape[0]
    param_kernels = {}
    for param_num, param_name in enumerate(parameter_names):
        kernel_name = parameter_names[param_num]
        param_kernels[kernel_name] = regression_results.coef_[param_num * shift_window_size:(param_num + 1) * shift_window_size]
    return param_kernels, shifts

In [152]:
plt.plot(save_kernels(param_names, results)[1], save_kernels(param_names, results)[0]['ipsi choices'])